In [ ]:
%matplotlib inline

OCCLUSION_SIZES = [ # occlusion size, occlusion stride
    [16, 16],
    [4, 2]
]

# Import the required modules
import nibabel as nib
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
import nibabel as nib
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage as ndi

heatmap = np.load('/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/I347770-4x4x4-s2_heatmap.npy')

print(heatmap.max())
print(heatmap.min())

# Dividimos por heatmap.max() porque los valores NO están entre 0 y 1 ya que son sumas.
# A su vez, no queremos normalizar, porque queremos mantener el cambio neutro en 0.
# heatmap /= max(heatmap.max(), -heatmap.min())

print(heatmap.shape)

# Convert the image data as a NumPy array
array = np.load('/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/I347770-4x4x4-s2_original.npy')

print(array.shape)

In [ ]:
def show_axial(heatmap, array, metadata, slice_threhsold = 0.5, show = True, save = False):
    heatmap = heatmap.copy()
    array = array.copy()
    
    # Display metadata information at the top
    metadata_str = f'Truth Label: {metadata["truthLabel"]}\n' \
                   f'Predicted Label: {metadata["predictedLabel"]}\n' \
                   f'Occlusion Stride: {metadata["occlusionStride"]}\n' \
                   f'Occlusion Size: {metadata["occlusionSize"]}\n' \
                   f'Study ID: {metadata["studyID"]}\n' \
                   f'Truth Label Category: {metadata["truthLabelCategory"]}\n\n'
    
    # Rotation
    for i in range(heatmap.shape[2]):  # Assuming axial slices are along the z-axis
        heatmap[:, :, i] = ndi.rotate(heatmap[:, :, i], 90, reshape=False)
    for i in range(array.shape[2]):  # Assuming axial slices are along the z-axis
        array[:, :, i] = ndi.rotate(array[:, :, i], 90, reshape=False)
        
    # Filter slices with values equal to 0
    non_zero_slices = np.any(array != -1, axis=(0, 1)) # The tensor is normalized -1, 1

    # Filter the image data
    array = array[:, :, non_zero_slices]

    # Filter the heatmap
    heatmap = heatmap[:, :, non_zero_slices]

    print(array.shape)
    
    # Filter slices with less than N occlusion change as max
    # threshold = slice_threhsold
    # thresholded_slices = np.any((heatmap > threshold) | (heatmap < - threshold), axis=(0, 1))
    # print(non_zero_slices)

    # Filter the image data
    # array = array[:, :, thresholded_slices]

    # Filter the heatmap
    # heatmap = heatmap[:, :, thresholded_slices]
    
    stride = 4 # Este debería ser el stride que usamos en el occlusion
    print(array.shape[2]//stride)
    fig, axs = plt.subplots(nrows=array.shape[2]//stride, ncols=2, figsize=(10, 5*(array.shape[2]//stride)))
    
    #fig.text(0.5, 0.95, metadata_str, horizontalalignment='center', verticalalignment='top', fontsize=10)
    
    fig.suptitle(metadata_str, fontsize=10, va='top')
    
    rang = max(heatmap.max(), -heatmap.min())
    
    for i, ax in enumerate(axs):
        # Left column: Image with heatmap overlay
        im = ax[0].imshow(array[:, :, i * stride], cmap='gray', origin='lower')
        im_overlay = ax[0].imshow(heatmap[:, :, i * stride], cmap='coolwarm', alpha=0.8, origin='lower', vmin=-rang, vmax=rang)
        ax[0].set_title(f'Axial Slice {i * stride}')

        # Add color bar for the heatmap overlay
        cbar = fig.colorbar(im_overlay, ax=ax[0], fraction=0.046, pad=0.04)
        cbar.set_label('Heatmap Intensity')

        # Right column: Original image
        ax[1].imshow(array[:, :, i * stride], cmap='gray', origin='lower')
        ax[1].set_title(f'Original Axial Slice {i * stride}')
        
    if save:
        plt.savefig(f"/Users/hugom/Tesis/Fleni/Occlusions/MedicalTransformer/{metadata['truthLabel']}-{metadata['studyID']}-occlusion{metadata['occlusionSize']}-stride{metadata['occlusionStride']}-axial", dpi=96)
        
    if show:
        # Adjust layout
        plt.tight_layout()
        # Show the plot
        plt.show()

AD: I347770

In [ ]:
metadata = {
    "truthLabel": "Label1",
    "predictedLabel": "Label2",
    "occlusionStride": 2,
    "occlusionSize": 4,
    "studyID": "123456",
    "truthLabelCategory": "visit953"
}

show_axial(heatmap, array, metadata, show = False, save = True)

In [ ]:
def show_coronal(heatmap, array, metadata, slice_threhsold = 0.5, show = True, save = False):
    heatmap = heatmap.copy()
    array = array.copy()
    
    # Rotation
    for i in range(heatmap.shape[1]):  # Assuming axial slices are along the z-axis
        heatmap[:, i, :] = ndi.rotate(heatmap[:, i, :], -90, reshape=False, mode = 'constant', cval = 0)
    for i in range(array.shape[1]):  # Assuming axial slices are along the z-axis
        array[:, i, :] = ndi.rotate(array[:, i, :], -90, reshape=False, mode = 'constant', cval = -1)
        
    # Filter slices with values equal to 0
    non_zero_slices = np.any(array != -1, axis=(0, 2)) # The tensor is normalized -1, 1

    # Filter the image data
    array = array[:, non_zero_slices, :]

    # Filter the heatmap
    heatmap = heatmap[:, non_zero_slices, :]

    print(array.shape)
    
    rang = max(heatmap.max(), -heatmap.min())
    
    # Filter slices with less than N occlusion change as max
    # threshold = slice_threhsold
    # thresholded_slices = np.any((heatmap > threshold) | (heatmap < - threshold), axis=(0, 2))
    # print(non_zero_slices)

    # Filter the image data
    # array = array[:, thresholded_slices, :]

    # Filter the heatmap
    # heatmap = heatmap[:, thresholded_slices, :]
    
    stride = 4 # Este debería ser el stride que usamos en el occlusion
    fig, axs = plt.subplots(nrows=array.shape[1]//stride, ncols=2, figsize=(10, 5*(array.shape[1]//stride)))
    for i, ax in enumerate(axs):
        # Left column: Image with heatmap overlay
        im = ax[0].imshow(array[:, i * stride, :], cmap='gray', origin='lower')
        im_overlay = ax[0].imshow(heatmap[:, i * stride, :], cmap='coolwarm', alpha=0.8, origin='lower', vmin=-rang, vmax=rang)
        ax[0].set_title(f'Axial Slice {i * stride}')

        # Add color bar for the heatmap overlay
        cbar = fig.colorbar(im_overlay, ax=ax[0], fraction=0.046, pad=0.04)
        cbar.set_label('Heatmap Intensity')

        # Right column: Original image
        ax[1].imshow(array[:, i * stride, :], cmap='gray', origin='lower')
        ax[1].set_title(f'Original Axial Slice {i * stride}')

    if save:
        plt.savefig(f"/Users/hugom/Tesis/Fleni/Occlusions/MedicalTransformer/{metadata['truthLabel']}-{metadata['studyID']}-occlusion{metadata['occlusionSize']}-stride{metadata['occlusionStride']}-coronal", dpi=96)
    
    if show:
        # Adjust layout
        plt.tight_layout()
        # Show the plot
        plt.show()

In [ ]:
show_coronal(heatmap, array, metadata, show = False, save = True)

In [ ]:
def show_saggital(heatmap, array, metadata, slice_threhsold = 0.5, show = True, save = False):
    heatmap = heatmap.copy()
    array = array.copy()

    # Rotation
    for i in range(heatmap.shape[0]):  # Assuming axial slices are along the z-axis
        heatmap[i, :, :] = ndi.rotate(heatmap[i, :, :], -90, reshape=False, mode = 'constant', cval = 0)
    for i in range(array.shape[0]):  # Assuming axial slices are along the z-axis
        array[i, :, :] = ndi.rotate(array[i, :, :], -90, reshape=False, mode = 'constant', cval = -1)
        
    # Filter slices with values equal to 0
    non_zero_slices = np.any(array != -1, axis=(1, 2)) # The tensor is normalized -1, 1

    # Filter the image data
    array = array[non_zero_slices, :, :]

    # Filter the heatmap
    heatmap = heatmap[non_zero_slices, :, :]

    print(array.shape)
    
    rang = max(heatmap.max(), -heatmap.min())
    
    # Filter slices with less than N occlusion change as max
    # threshold = slice_threhsold
    # thresholded_slices = np.any((heatmap > threshold) | (heatmap < - threshold), axis=(1, 2))
    # print(non_zero_slices)

    # Filter the image data
    # array = array[thresholded_slices, :, :]

    # Filter the heatmap
    # heatmap = heatmap[thresholded_slices, :, :]
    
    stride = 4 # Este debería ser el stride que usamos en el occlusion
    fig, axs = plt.subplots(nrows=array.shape[0]//stride, ncols=2, figsize=(10, 5*(array.shape[0]//stride)))
    for i, ax in enumerate(axs):
        # Left column: Image with heatmap overlay
        im = ax[0].imshow(array[i * stride, :, :], cmap='gray', origin='lower')
        im_overlay = ax[0].imshow(heatmap[i * stride, :, :], cmap='coolwarm', alpha=0.8, origin='lower', vmin=-rang, vmax=rang)
        ax[0].set_title(f'Axial Slice {i * stride}')

        # Add color bar for the heatmap overlay
        cbar = fig.colorbar(im_overlay, ax=ax[0], fraction=0.046, pad=0.04)
        cbar.set_label('Heatmap Intensity')

        # Right column: Original image
        ax[1].imshow(array[i * stride, :, :], cmap='gray', origin='lower')
        ax[1].set_title(f'Original Axial Slice {i * stride}')

    if save:
        plt.savefig(f"/Users/hugom/Tesis/Fleni/Occlusions/MedicalTransformer/{metadata['truthLabel']}-{metadata['studyID']}-occlusion{metadata['occlusionSize']}-stride{metadata['occlusionStride']}-sagittal", dpi=96)
    
    if show:
        # Adjust layout
        plt.tight_layout()
        # Show the plot
        plt.show()

In [ ]:
show_saggital(heatmap, array, metadata)

# CN

In [ ]:
%matplotlib inline
# Load the NIfTI image

# Import the required modules
import nibabel as nib
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
import nibabel as nib
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage as ndi

heatmap = np.load('/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/I77185-4x4x4-s2_heatmap.npy')

print(heatmap.max())
print(heatmap.min())

# Dividimos por heatmap.max() porque los valores NO están entre 0 y 1 ya que son sumas.
# A su vez, no queremos normalizar, porque queremos mantener el cambio neutro en 0.
# heatmap /= max(heatmap.max(), -heatmap.min())


print(heatmap.shape)

# Convert the image data as a NumPy array
array = np.load('/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/I77185-4x4x4-s2_original.npy')

print(array.shape)

In [ ]:
show_axial(heatmap, array, metadata)

In [ ]:
show_coronal(heatmap, array, metadata)

In [ ]:
show_saggital(heatmap, array, metadata)

In [ ]:
# El confidence en este caso es del modelo nuevo
ads = [{
    "studyID": "I128833",
    "confidence": 1,
    "predictedLabel": "AD",
}, {
    "studyID": "I47403",
    "confidence": 0.9483,
    "predictedLabel": "AD",
}, {
    "studyID": "I146553",
    "confidence": 0.99998,
    "predictedLabel": "AD",
}, {
    "studyID": "I1334685",
    "confidence": 0.9397,
    "predictedLabel": "AD",
}, {
    "studyID": "I85844",
    "confidence": 1,
    "predictedLabel": "AD",
}]
noAds = [{
    "studyID": "I225971",
    "confidence": 0.7502,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I321658",
    "confidence": 0.9641,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I77185",
    "confidence": 0.99996,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I325148",
    "confidence": 0.9952,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I223643",
    "confidence": 0.9954,
    "predictedLabel": "no-AD",
}]

In [ ]:
def showAllMedicalTransformerImages(lst, category, occlusionSize = 4, occlusionStride = 2):
    for imageData in lst:
        studyID = imageData['studyID']
        print(f"Processing {studyID}")
        metadata = {
            "occlusionSize": occlusionSize,
            "occlusionStride": occlusionStride,
            "truthLabel": category,
            "truthLabelCategory": "visit953"
        }
        metadata = dict(list(metadata.items()) + list(imageData.items()))
        heatmap = np.load(f"/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/{studyID}-size{occlusionSize}-stride{occlusionStride}_heatmap.npy")

        if heatmap.max() == heatmap.min():
            print("Warning: empty heatmap")
            
        if category != "AD":
            heatmap *= -1

        # Convert the image data as a NumPy array
        array = np.load(f"/Users/hugom/PET-IA/MedicalTransformer/2_Pretraining_Masked_Encoding_Vector_Prediction/occlusions/{studyID}-size{occlusionSize}-stride{occlusionStride}_original.npy")
        
        show_axial(heatmap, array, metadata, save = True, show = False)
        show_coronal(heatmap, array, metadata, save = True, show = False)
        show_saggital(heatmap, array, metadata, save = True, show = False)

In [ ]:
# ejecutar en loop para crear todas las imágenes
showAllMedicalTransformerImages(ads, "AD", 4, 2)
showAllMedicalTransformerImages(ads, "AD", 16, 16)

In [ ]:
showAllMedicalTransformerImages(noAds, "no-AD", 4, 2)
showAllMedicalTransformerImages(noAds, "no-AD", 16, 16)

# Modelo viejo

no-AD: I77185

In [ ]:
%matplotlib inline

heatmap = np.load('/Users/hugom/PET-IA/src/I77185_heatmap.npy')

print(heatmap.max())
print(heatmap.min())

print(heatmap.shape)

# Load the Nifti File
# image = nib.load(nifti_path)

# Convert the image data as a NumPy array
array = np.load('/Users/hugom/PET-IA/src/I77185_original.npy')
print(array.min())
print(array.max())

print(array.shape)

In [ ]:
def show_axial_yiming(heatmap, array, metadata, heatmap_axis = 0, save = False):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    # Left column: Image with heatmap overlay
    im = axs[0].imshow(array[0, :, :], cmap='gray', origin='lower')
    rang = max(heatmap.max(), -heatmap.min())
    im_overlay = axs[0].imshow(heatmap[heatmap_axis, :, :], cmap='coolwarm', alpha=0.5, origin='lower', vmin = -rang, vmax = rang)
    axs[0].set_title(f'Axial Slice')

    # Add color bar for the heatmap overlay
    cbar = fig.colorbar(im_overlay, ax=axs[0])
    cbar.set_label('Heatmap Intensity')
    
    # Display metadata information
    metadata_str = f'Truth Label: {metadata["truthLabel"]}\n' \
                   f'Predicted Label: {metadata["predictedLabel"]}\n' \
                   f'Confidence: {metadata["confidence"]}\n' \
                   f'Occlusion Stride: {metadata["occlusionStride"]}\n' \
                   f'Occlusion Size: {metadata["occlusionSize"]}\n' \
                   f'Study ID: {metadata["studyID"]}\n' \
                   f'Truth Label Category: {metadata["truthLabelCategory"]}'
    
    axs[0].text(0, 1.5, metadata_str, transform=axs[0].transAxes, verticalalignment='center', fontsize=10)

    # Right column: Original image
    axs[1].imshow(array[0, :, :], cmap='gray', origin='lower')
    axs[1].set_title(f'Original Axial Slice')

    # Adjust layout
    plt.tight_layout()
    
    if save:
        plt.savefig(f"/Users/hugom/Tesis/Fleni/Occlusions/Yiming/{metadata['truthLabel']}-{metadata['studyID']}-o{metadata['occlusionSize']}-s{metadata['occlusionStride']}", dpi=300)

    
    # Show the plot
    plt.show()

In [ ]:
metadata = {
    "truthLabel": "no-AD",
    "predictedLabel": "no-AD",
    "confidence": "?",
    "occlusionStride": 16,
    "occlusionSize": 16,
    "studyID": "I77185",
    "truthLabelCategory": "visit953"
}

show_axial_yiming(heatmap, array, metadata, 0)

AD: I347770

In [ ]:
metadata = {
    "truthLabel": "AD",
    "predictedLabel": "AD",
    "occlusionStride": 16,
    "occlusionSize": 16,
    "confidence": "?",
    "studyID": "I347770",
    "truthLabelCategory": "visit953"
}

In [ ]:
heatmap = np.load('/Users/hugom/PET-IA/src/I347770_heatmap.npy')

print(heatmap.max())
print(heatmap.min())

print(heatmap.shape)

# Load the Nifti File
# image = nib.load(nifti_path)

# Convert the image data as a NumPy array
array = np.load('/Users/hugom/PET-IA/src/I347770_original.npy')
print(array.min())
print(array.max())

print(array.shape)

In [ ]:
show_axial_yiming(heatmap, array, metadata, 0)

El modelo parece estar sesgado hacia AD. Es decir, predice AD por defecto excepto que encuentre algo que lo haga pensar que NO es AD?

# Todo en un for

In [ ]:
ads = ["I128833", "I47403", "I146553", "I1334685", "I85844"]
ads = [{
    "studyID": "I128833",
    "confidence": 1,
    "predictedLabel": "AD",
}, {
    "studyID": "I47403",
    "confidence": 0.9985,
    "predictedLabel": "AD",
}, {
    "studyID": "I146553",
    "confidence": 0.9980,
    "predictedLabel": "AD",
}, {
    "studyID": "I1334685",
    "confidence": 0.9981,
    "predictedLabel": "AD",
}, {
    "studyID": "I85844",
    "confidence": 0.9999,
    "predictedLabel": "AD",
}]
noAds = ["I225971", "I321658", "I77185", "I325148", "I223643"]
noAds = [{
    "studyID": "I225971",
    "confidence": 1,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I321658",
    "confidence": 0.99963,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I77185",
    "confidence": 1,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I325148",
    "confidence": 0.9999,
    "predictedLabel": "no-AD",
}, {
    "studyID": "I223643",
    "confidence": 0.99999,
    "predictedLabel": "no-AD",
}]

## ADs

In [ ]:
def showAllYimingImages(lst, category):
    for imageData in lst:
        studyID = imageData['studyID']
        metadata = {
            "occlusionStride": 16,
            "occlusionSize": 16,
            "truthLabel": category,
            "truthLabelCategory": "visit953"
        }
        metadata = dict(list(metadata.items()) + list(imageData.items()))
        heatmap = np.load(f"/Users/hugom/PET-IA/src/{studyID}_heatmap.npy")

        if heatmap.max() == heatmap.min():
            print("Warning: empty heatmap")
            
        if category != "AD":
            heatmap *= -1

        # Convert the image data as a NumPy array
        array = np.load(f"/Users/hugom/PET-IA/src/{studyID}_original.npy")
        
        show_axial_yiming(heatmap, array, metadata, 0, save = True)

In [ ]:
showAllYimingImages(ads, "AD",)

# no-ADs

In [ ]:
showAllYimingImages(noAds, "no-AD")